In [5]:
import pandas as pd

cbb3 = pd.read_csv('cbb_trial1.csv', encoding='latin-1')
cbb3['TEAM'] = cbb3['TEAM'].str.replace('Ê', '')
cbb3['REC'] = cbb3['REC'].str.replace('Ð', '–')
cbb3 = cbb3.drop(cbb3.index[10695:], axis=0)
cbb3.reset_index(drop=True, inplace=True)
cbb3['SEED'] = None
cbb3['FINISH'] = None

# Iterate through the DataFrame
for index, row in cbb3.iterrows():
    if pd.notna(row['TEAM']) and 'seed' in row['TEAM']:
        # In this part, we extract SEED and FINISH information
        seed_part = row['TEAM'].split(' seed')[0].lstrip('Ê')  # We were getting an error saying we still had Ê before some integers, so we remove them with this line
        seed = int(seed_part) if seed_part.isdigit() else None  # Convert to integer, handling non-numeric values
        finish = row['TEAM'].split(',')[1]
        
        # Here, we update the values in the row above
        if index > 0:
            cbb3.at[index - 1, 'SEED'] = seed
            cbb3.at[index - 1, 'FINISH'] = finish

cbb3.reset_index(drop=True, inplace=True)
cbb3.drop('BARTHAG', axis=1, inplace=True)
cbb3[['WINS', 'LOSSES']] = cbb3['REC'].str.split('–', expand=True).apply(pd.to_numeric, errors='coerce')
cbb3['G'] = pd.to_numeric(cbb3['G'], errors='coerce')
cbb3.loc[:, 'ADJOE':'WAB'] = cbb3.loc[:, 'ADJOE':'WAB'].apply(pd.to_numeric, errors='coerce')
cbb3['SEED'] = pd.to_numeric(cbb3['SEED'], errors='coerce')
cbb3.to_csv('cbb_edited.csv')


In [43]:
df = pd.read_csv('cbb_edited.csv')
df = df.drop('Unnamed: 0', axis=1)
df = df.dropna(subset=['YEAR'])
df = df.dropna(subset=['FINISH'])
df['FINISH'] = df['FINISH'].str.strip()
df[df['FINISH'].astype(str) == 'R68']
filter = df['FINISH'] != 'R68'
df = df[filter]
df.to_csv('March_Madness.csv')